<div align="center">
# 🔐 Projet Mastercamp 2025  
## Surveillance & Analyse Automatisée des Vulnérabilités ANSSI (CVE)

**Participants :**  
- Abouleila Selim  
- Sozer Begumu  
- Nguedia Duval  
- Sami Samiali  

</div>


</div>


# Étape 1 : Récupération des liens des bulletins ANSSI

## Objectif
Collecter automatiquement les URLs des bulletins d’alerte et d’avis publiés par l’ANSSI pour alimenter la veille de vulnérabilités)
html = response.text


In [19]:
import feedparser

url = "https://www.cert.ssi.gouv.fr/avis/feed/"
rss_feed = feedparser.parse(url)

for entry in rss_feed.entries:
    print(entry.title, entry.link, entry.published, entry.description)

Multiples vulnérabilités dans Mattermost Server (13 mai 2025) https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-AVI-0392/ Tue, 13 May 2025 00:00:00 +0000 De multiples vulnérabilités ont été découvertes dans Mattermost Server. Elles permettent à un attaquant de provoquer une atteinte à la confidentialité des données et un contournement de la politique de sécurité.
Multiples vulnérabilités dans les produits SAP (13 mai 2025) https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-AVI-0396/ Tue, 13 May 2025 00:00:00 +0000 De multiples vulnérabilités ont été découvertes dans les produits SAP. Certaines d'entre elles permettent à un attaquant de provoquer une exécution de code arbitraire à distance, une atteinte à la confidentialité des données et une injection de code indirecte à distance (XSS).
Vulnérabilité dans Roundcube (02 juin 2025) https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-AVI-0468/ Mon, 02 Jun 2025 00:00:00 +0000 Une vulnérabilité a été découverte dans Roundcube Roundcube Webmail. Elle perme

In [21]:
import feedparser
url2 = "https://www.cert.ssi.gouv.fr/alerte/feed"
rss_alerte = feedparser.parse(url2)

for entry in rss_alerte.entries:
    print("Title:", entry.title)
    print("Link:", entry.link)
    print("Published:", entry.published)
    print("Description:", entry.description)
    print("-" * 20) # Print a separator line

Title: Multiples vulnérabilités dans Microsoft Windows (16 septembre 2022)
Link: https://www.cert.ssi.gouv.fr/alerte/CERTFR-2022-ALE-007/
Published: Fri, 16 Sep 2022 00:00:00 +0000
Description: \[MàJ du 31 octobre 2022\] Une preuve de concept est publiquement disponible. \[Publication initiale\] Dans le cadre de son *Patch Tuesday*, en date du 13 septembre 2022, Microsoft a indiqué l'existence de multiples vulnérabilités au sein de plusieurs versions de Windows. Trois d'entre elles...
--------------------
Title: [MaJ] Multiples vulnérabilités dans Microsoft Exchange (30 septembre 2022)
Link: https://www.cert.ssi.gouv.fr/alerte/CERTFR-2022-ALE-008/
Published: Fri, 30 Sep 2022 00:00:00 +0000
Description: \[Mise à jour du 09 novembre 2022\] L'éditeur a publié un correctif (cf. section solution). En date du 29 septembre 2022, Microsoft a indiqué l'existence de deux vulnérabilités, de type zéro-jour, au sein de Windows Exchange 2013, 2016 et 2019. Ces vulnérabilités sont les suivantes : -..

# Étape 2 : Extraction des CVE

## Objectif
Extraire automatiquement les identifiants CVE (« Common Vulnerabilities and Exposures ») à partir du JSON de chaque bulletin ANSSI.

In [89]:
# -----------------------------------------------------------
# 1) Imports + where the raw files live
# -----------------------------------------------------------
from pathlib import Path
import json, re, html
import pandas as pd
from json import JSONDecodeError

ALERTE_DIR = Path(r"C:\Users\alpha\Documents\GitHUB_Projects\Atelier_Data_Porject\Atelier_Data_Project\data_pour_TD_final\alertes")
AVIS_DIR   = Path(r"C:\Users\alpha\Documents\GitHUB_Projects\Atelier_Data_Porject\Atelier_Data_Project\data_pour_TD_final\Avis")

# -----------------------------------------------------------
# 2) Helpers
# -----------------------------------------------------------
def strip_md(text: str | None) -> str:
    """Drop HTML/Markdown – keep plain text."""
    if not text:
        return ""
    text = re.sub(r"<[^>]+>", "", text)        # HTML tags
    text = re.sub(r"[*_`#>-]", "", text)       # basic MD chars
    return html.unescape(text).strip()


def read_json(path: Path) -> dict | None:
    """Try UTF-8 first, then cp1252; return None if not JSON."""
    for enc in ("utf-8", "cp1252"):
        try:
            return json.loads(path.read_text(encoding=enc))
        except (UnicodeDecodeError, JSONDecodeError):
            continue
    return None


def record_from_obj(obj: dict, kind: str) -> dict:
    """Flatten one CERT-FR object into a flat dict ready for DataFrame."""
    return {
        "reference": obj.get("reference"),
        "title":     obj.get("title"),
        "closed_at": obj.get("closed_at") if kind == "alerte" else "",
        "cves":      "; ".join(c["name"] for c in obj.get("cves", [])) or "Aucune",
        "risks":     "; ".join(r["description"] for r in obj.get("risks", [])) or "Non précisé",
        "summary":   strip_md(obj.get("summary"))[:1500],   # trim long blobs
        "first_revision": (obj.get("revisions") or [{}])[0].get("revision_date", ""),
        "affected_systems": " | ".join(
            f'{s.get("product", {}).get("vendor", {}).get("name","?")} '
            f'{s.get("product", {}).get("name","?")} – {s.get("description","")}'
            for s in obj.get("affected_systems", [])
        ),
    }

# -----------------------------------------------------------
# 3) Walk the folders, parse everything
# -----------------------------------------------------------
def build_df(folder: Path, kind: str) -> pd.DataFrame:
    rows, bad = [], []
    for f in folder.rglob("*"):                 # recurse (works with nested folders)
        if not f.is_file():
            continue
        obj = read_json(f)
        if obj is None:
            bad.append(f.name)
            continue
        rows.append(record_from_obj(obj, kind))
    if bad:
        print(f"⚠️  {len(bad)} files in '{folder.name}' skipped (non-JSON or unreadable)")
    return pd.DataFrame(rows)

alerte_df = build_df(ALERTE_DIR, "alerte")
avis_df   = build_df(AVIS_DIR,   "avis")


# -----------------------------------------------------------
# 4) Write the spreadsheets & preview
# -----------------------------------------------------------
alerte_df.to_csv("alertes.csv", index=False, encoding="utf-8")
avis_df.to_csv("avis.csv",     index=False, encoding="utf-8")

print(f"✅  {len(alerte_df):>4} alertes → alertes.csv")
print(f"✅  {len(avis_df):>4} avis    → avis.csv")

display(alerte_df.head())
display(avis_df.head())



✅    72 alertes → alertes.csv
✅  2673 avis    → avis.csv


,reference,title,closed_at,cves,risks,summary,first_revision,affected_systems
0,CERTFR-2021-ALE-001,|MàJ] Vulnérabilité dans SonicWall SMA100,2021-05-12,CVE-2021-20016; CVE-2014-6271,Exécution de code arbitraire à distance,\[Mise à jour du 30 avril 2021\]\n\nLe 29 avri...,2021-02-02T01:00:00.000000,"Sonicwall Secure Mobile Access – SMA 200, SMA ..."
1,CERTFR-2021-ALE-002,[MàJ] Vulnérabilité dans Google Chrome et Micr...,2021-03-11,CVE-2021-21148,Non spécifié par l'éditeur,\[Mise à jour du 08 février 2021\]\n\nLe navig...,2021-02-05T01:00:00.000000,Microsoft Edge – Microsoft Edge (Chromium-base...
2,CERTFR-2021-ALE-003,[MàJ] Vulnérabilité dans VMware vCenter Server,2021-05-12,CVE-2021-21972,Exécution de code arbitraire à distance,\[version du 26 février 2021\]\n\nLe CERTFR a ...,2021-02-25T01:00:00.000000,VMware vCenter Server – Cloud Foundation (vCen...
3,CERTFR-2021-ALE-004,[MàJ] Multiples vulnérabilités dans Microsoft ...,2021-07-16,CVE-2021-26858; CVE-2021-27078; CVE-2021-26857...,Exécution de code arbitraire à distance,\[version du 16 mars 2021\]\n\nle 15 mars 2021...,2021-03-03T01:00:00.000000,
4,CERTFR-2021-ALE-005,Multiples vulnérabilités dans Microsoft DNS se...,2021-05-12,CVE-2021-26893; CVE-2021-26897; CVE-2021-26894...,Exécution de code arbitraire à distance,Contexte\n\nSept vulnérabilités concernant le ...,2021-03-12T01:00:00.000000,


,reference,title,closed_at,cves,risks,summary,first_revision,affected_systems
0,CERTFR-2023-AVI-0001,Vulnérabilité dans Apache Tomcat,,CVE-2022-45143,Non spécifié par l'éditeur,Une vulnérabilité a été découverte dans Apache...,2023-01-04T01:00:00.000000,Apache Tomcat – Apache Tomcat versions 9.x ant...
1,CERTFR-2023-AVI-0002,Multiples vulnérabilités dans les produits For...,,CVE-2022-45857; CVE-2022-35845; CVE-2022-39947...,Injection de code indirecte à distance (XSS); ...,De multiples vulnérabilités ont été corrigées ...,2023-01-04T01:00:00.000000,Fortinet FortiADC – FortiADC versions 7.x anté...
2,CERTFR-2023-AVI-0003,Multiples vulnérabilités dans les produits And...,,CVE-2022-22088; CVE-2022-42720; CVE-2022-44436...,Déni de service à distance; Exécution de code ...,De multiples vulnérabilités ont été découverte...,2023-01-04T01:00:00.000000,"Google Android – Android 10, 11, 12, 12L, 13 s..."
3,CERTFR-2023-AVI-0004,Multiples vulnérabilités dans IBM Sterling Glo...,,CVE-2021-44521; CVE-2020-10663,Exécution de code arbitraire à distance; Conto...,De multiples vulnérabilités ont été découverte...,2023-01-04T01:00:00.000000,IBM Sterling – IBM Sterling Global Mailbox ver...
4,CERTFR-2023-AVI-0005,Vulnérabilité dans Synology VPN Plus Server,,CVE-2022-43931,Exécution de code arbitraire à distance,Une vulnérabilité a été découverte dans Synolo...,2023-01-04T01:00:00.000000,Synology SRM – VPN Plus Server pour SRM 1.3 ve...


In [93]:
# -----------------------------------------------------------
# 5) Rassembler tous les identifiants CVE en une seule liste
# -----------------------------------------------------------
import re

# concatène les deux colonnes cves, les nettoie et garde le bon format
cve_ids = (
    pd.concat([alerte_df["cves"], avis_df["cves"]])  # une seule série
      .dropna()                                     # enlève les NaN
      .str.split(";")                               # sépare "CVE1; CVE2"
      .explode()                                    # une ligne par CVE
      .str.strip()                                  # espaces
      .loc[lambda s: s.str.match(r"^CVE-\d{4}-\d{4,}$")]  # filtre vrai format
      .unique()                                     # dé-duplication
      .tolist()                                     # -> liste Python
)

print(f"📌 {len(cve_ids)} identifiants CVE collectés")

# (optionnel) écrire la liste dans un fichier texte pour vérification
with open("cve_ids.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(cve_ids))
print("📝  Liste enregistrée dans cve_ids.txt")


📌 22309 identifiants CVE collectés
📝  Liste enregistrée dans cve_ids.txt


# Étape 3 : Enrichissement des CVE

## Objectif  
Pour chaque identifiant CVE extrait, récupérer automatiquement ses métadonnées (description, scores CVSS, date de publication, sources externes, EPSS…) afin de pouvoir prioriser et consolider la veille.


In [101]:
# -----------------------------------------------------------
# 6) Interroger les API MITRE CVE AWG + EPSS pour chaque CVE
# -----------------------------------------------------------
import requests, json, time
import pandas as pd
from pathlib import Path

# 1) Lire la liste des CVE collectés à l’étape 5
with open("cve_ids.txt", encoding="utf-8") as f:
    cve_list = [ln.strip() for ln in f if ln.strip()]


cve_list = cve_list[:1000]          # ← limite aux 1 000 premiers identifiants

print(f"🔍  Interrogation de {len(cve_list)} identifiants CVE")

# 2) Préparer une structure pour stocker tous les résultats
records = []

# 3) Boucle principale — le code interne est VOTRE exemple inchangé
for cve_id in cve_list:
    try:
        # ================= MITRE CVE AWG =================
        url = f"https://cveawg.mitre.org/api/cve/{cve_id}"
        response = requests.get(url, timeout=15)
        data = response.json()           # <-- peut lever ValueError si pas du JSON

        # Extraire la description
        description = data["containers"]["cna"]["descriptions"][0]["value"]

        # Extraire le score CVSS (prise en compte de toutes les variantes)
        cvss_score = ""
        try:
            cvss_score = data["containers"]["cna"]["metrics"][0]["cvssV3_1"]["baseScore"]
        except KeyError:
            try:
                cvss_score = data["containers"]["cna"]["metrics"][0]["cvssV3_0"]["baseScore"]
            except KeyError:
                pass                        # le champ n’existe pas → reste vide

        # Extraire le CWE
        cwe       = "Non disponible"
        cwe_desc  = "Non disponible"
        problemtype = data["containers"]["cna"].get("problemTypes", [])
        if problemtype and problemtype[0].get("descriptions"):
            cwe       = problemtype[0]["descriptions"][0].get("cweId", "Non disponible")
            cwe_desc  = problemtype[0]["descriptions"][0].get("description", "Non disponible")

        # Extraire les produits affectés
        affected_str = ""
        affected = data["containers"]["cna"].get("affected", [])
        if affected:
            lines = []
            for product in affected:
                vendor        = product.get("vendor", "Inconnu")
                product_name  = product.get("product", "Inconnu")
                versions = [
                    v["version"]
                    for v in product.get("versions", [])
                    if v.get("status") == "affected"
                ]
                lines.append(f"{vendor} | {product_name} | {', '.join(versions)}")
            affected_str = " || ".join(lines)

        # ================ EPSS ============================
        epss_score = ""
        try:
            url_epss = f"https://api.first.org/data/v1/epss?cve={cve_id}"
            epss_resp = requests.get(url_epss, timeout=10)
            epss_json = epss_resp.json()
            epss_data = epss_json.get("data", [])
            if epss_data:
                epss_score = epss_data[0]["epss"]
        except Exception:
            pass  # on ignore les échecs EPSS

        # Affichage (conserve exactement vos prints)
        print(f"\nCVE : {cve_id}")
        print(f"Description : {description}")
        print(f"Score CVSS : {cvss_score}")
        print(f"Type CWE : {cwe}")
        print(f"CWE Description : {cwe_desc}")
        print(f"Score EPSS : {epss_score}")
        if affected_str:
            print(f"Produits affectés : {affected_str}")

        # Mémoriser dans la table finale
        records.append(
            {
                "cve":            cve_id,
                "description":    description,
                "cvss_score":     cvss_score,
                "cwe":            cwe,
                "cwe_description": cwe_desc,
                "epss_score":     epss_score,
                "affected":       affected_str,
            }
        )

        time.sleep(0.001)  # petite pause pour ne pas surcharger les API

    except Exception as e:
        print(f"\n⚠️  Problème avec {cve_id} → {e}")

# 4) Exporter la récolte dans un CSV
out_df = pd.DataFrame(records)
out_df.to_csv("cve_enrichissement_api.csv", index=False, encoding="utf-8")
print(f"\n  Fichier 'cve_enrichissement_api.csv' écrit ({len(out_df)} lignes)")


🔍  Interrogation de 1000 identifiants CVE

CVE : CVE-2021-20016
Description : A SQL-Injection vulnerability in the SonicWall SSLVPN SMA100 product allows a remote unauthenticated attacker to perform SQL query to access username password and other session related information. This vulnerability impacts SMA100 build version 10.x.
Score CVSS : 
Type CWE : CWE-89
CWE Description : CWE-89: Improper Neutralization of Special Elements used in an SQL Command ('SQL Injection')
Score EPSS : 0.804450000
Produits affectés : SonicWall | SonicWall SMA100 | SMA100 build version 10.x

CVE : CVE-2014-6271
Description : GNU Bash through 4.3 processes trailing strings after function definitions in the values of environment variables, which allows remote attackers to execute arbitrary code via a crafted environment, as demonstrated by vectors involving the ForceCommand feature in OpenSSH sshd, the mod_cgi and mod_cgid modules in the Apache HTTP Server, scripts executed by unspecified DHCP clients, and oth

# Étape 4 : Consolidation des Données

## Objectif  
Assembler en un seul tableau (DataFrame Pandas) l’ensemble des métadonnées ANSSI et CVE enrichies, pour pouvoir filtrer, trier et exporter facilement.

## Structure du DataFrame  
| Colonne                 | Description                                                    |
|-------------------------|----------------------------------------------------------------|
| **ID ANSSI**            | Identifiant du bulletin (ex. `CERTFR-2024-ALE-001`)            |
| **Titre ANSSI**         | Titre de l’avis ou de l’alerte                                 |
| **Type**                | `'Alerte'` ou `'Avis'`                                         |
| **Date**                | Date de publication (YYYY-MM-JJ)                               |
| **Lien**                | URL du bulletin ANSSI                                          |
| **CVE**                 | Identifiant de la vulnérabilité                                |
| **CVSS**                | Score CVSS (0–10)                                              |
| **Base Severity**       | Gravité associée (Faible, Moyenne, Élevée, Critique)          |
| **CWE**                 | Code CWE (ex. `CWE-79`)                                        |
| **EPSS**                | Score EPSS (0–1)                                               |
| **Description**         | Description détaillée issue des API                            |
| **Éditeur**             | Vendor du produit affecté                                      |
| **Produit**             | Nom du produit                                                 |
| **Versions affectées**  | Chaîne de versions impactées (séparées par des virgules)       |

## Exemple de DataFrame  
| ID ANSSI             | Titre ANSSI                         | Type   | Date       | CVE              | CVSS | Base Severity | CWE    | EPSS  | Lien                                  | Description                                   | Éditeur | Produit | Versions affectées   |
|----------------------|--------------------------------------|--------|------------|------------------|------|---------------|--------|-------|---------------------------------------|-----------------------------------------------|---------|---------|----------------------|
| CERTFR-2024-ALE-001  | Multiples vulnérabilités dans Ivanti | Alerte | 2024-01-11 | CVE-2024-49126   | 9.0  | Critique      | CWE-287| 0.85  | https://…/CERTFR-2024-ALE-001/json/   | Authentication bypass…                        | Ivanti  | ICS     | 9.1R18, 22.6R2       |
| CERTFR-2024-ALE-001  | Multiples vulnérabilités dans Ivanti | Alerte | 2024-01-11 | CVE-2023-46805   | 8.2  | Élevée        | —      | 0.94  | https://…/CERTFR-2024-ALE-001/json/   | Authentication bypass…                        | Ivanti  | IPS     | 9.1R18, 22.6R1       |

In [91]:
# Importations
import requests
import pandas as pd
from IPython.display import display

# API endpoints
MITRE_API_BASE = "https://cveawg.mitre.org/api/cve"
EPSS_API_BASE  = "https://api.first.org/data/v1/epss"

# Fonctions d’enrichissement

def fetch_mitre_cve(cve_id):
    """Récupère description, score CVSS, CWE et produits affectés via l’API MITRE."""
    result = {"cve_id": cve_id}
    try:
        resp = requests.get(f"{MITRE_API_BASE}/{cve_id}")
        resp.raise_for_status()
        data = resp.json()["containers"]["cna"]
    except Exception as e:
        result.update({
            "description": f"Erreur MITRE: {e}",
            "cvss_score": None,
            "cwe": None,
            "cwe_desc": None,
            "affected": []
        })
        return result

    # Description
    descs = data.get("descriptions", [])
    result["description"] = descs[0]["value"] if descs else "Non disponible"

    # Score CVSS v3.x
    result["cvss_score"] = None
    for metric in data.get("metrics", []):
        if "cvssV3_1" in metric:
            result["cvss_score"] = metric["cvssV3_1"].get("baseScore")
            break
        if "cvssV3_0" in metric:
            result["cvss_score"] = metric["cvssV3_0"].get("baseScore")
            break

    # Type CWE
    result["cwe"] = None
    result["cwe_desc"] = None
    ptypes = data.get("problemTypes", [])
    if ptypes and "descriptions" in ptypes[0]:
        entry = ptypes[0]["descriptions"][0]
        result["cwe"] = entry.get("cweId")
        result["cwe_desc"] = entry.get("description")

    # Produits affectés
    result["affected"] = []
    for prod in data.get("affects", {}).get("vendor", []):
        vendor = prod.get("vendor_name", "N/A")
        for product in prod.get("product", []):
            name = product.get("product_name", "N/A")
            versions = []
            for vers in product.get("version", []):
                for status_block in vers.get("status", []):
                    if status_block.get("status") == "affected":
                        versions.append(vers.get("version_value"))
            result["affected"].append({
                "vendor": vendor,
                "product": name,
                "versions": versions or ["N/A"]
            })

    return result

def fetch_epss(cve_id):
    """Récupère le score EPSS via l’API FIRST."""
    try:
        resp = requests.get(f"{EPSS_API_BASE}?cve={cve_id}")
        resp.raise_for_status()
        data = resp.json().get("data", [])
        if data and "epss" in data[0]:
            return float(data[0]["epss"])
    except Exception:
        pass
    return None

def get_severity_label(score):
    """Traduit un score CVSS en libellé de gravité."""
    if score is None:
        return "Non disponible"
    if score <= 3:
        return "Faible"
    if score <= 6:
        return "Moyenne"
    if score <= 8:
        return "Élevée"
    return "Critique"

# Liste de CVE à traiter
cve_list = [
    "CVE-2023-24488",
    "CVE-2023-46805",
    "CVE-2024-21887",
    "CVE-2024-21893",
    "CVE-2024-21888",
    "CVE-2024-22024",
]

# Collecte des données
rows = []
for cve_id in cve_list:
    rec = fetch_mitre_cve(cve_id)
    rec["epss_score"] = fetch_epss(cve_id)
    # Si pas de affected, on ajoute une ligne vide
    if not rec.get("affected"):
        rows.append({
            "CVE": rec["cve_id"],
            "Description": rec["description"],
            "CVSS": rec["cvss_score"],
            "Gravité": get_severity_label(rec["cvss_score"]),
            "CWE": rec["cwe"] or "Non disponible",
            "EPSS": rec["epss_score"],
            "Éditeur": None,
            "Produit": None,
            "Versions affectées": None
        })
    else:
        for p in rec["affected"]:
            rows.append({
                "CVE": rec["cve_id"],
                "Description": rec["description"],
                "CVSS": rec["cvss_score"],
                "Gravité": get_severity_label(rec["cvss_score"]),
                "CWE": rec["cwe"] or "Non disponible",
                "EPSS": rec["epss_score"],
                "Éditeur": p["vendor"],
                "Produit": p["product"],
                "Versions affectées": ", ".join(p["versions"])
            })

# Création et affichage du DataFrame
df = pd.DataFrame(rows)
display(df)


,CVE,Description,CVSS,Gravité,CWE,EPSS,Éditeur,Produit,Versions affectées
0,CVE-2023-24488,Cross site scripting vulnerability in Citrix A...,6.1,Élevée,CWE-79,0.91357,None,None,None
1,CVE-2023-46805,An authentication bypass vulnerability in the ...,8.2,Critique,Non disponible,0.94398,None,None,None
2,CVE-2024-21887,A command injection vulnerability in web compo...,9.1,Critique,Non disponible,0.94416,None,None,None
3,CVE-2024-21893,A server-side request forgery vulnerability in...,8.2,Critique,Non disponible,0.94320,None,None,None
4,CVE-2024-21888,A privilege escalation vulnerability in web co...,8.8,Critique,Non disponible,0.64798,None,None,None
5,CVE-2024-22024,An XML external entity or XXE vulnerability in...,8.3,Critique,Non disponible,0.94320,None,None,None


In [66]:
# ─────────────────────────────────────────────────────────────────────────────
# VISUALISATIONS DEMANDÉES – à exécuter quand le DataFrame df existe déjà
# ─────────────────────────────────────────────────────────────────────────────
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# 1) Garantir que les colonnes numériques le sont vraiment
df["CVSS_num"] = pd.to_numeric(df["CVSS"], errors="coerce")
df["EPSS_num"] = pd.to_numeric(df["EPSS"], errors="coerce")

# -----------------------------------------------------------------------------
# A. Histogramme des scores CVSS (avec gravité)
# -----------------------------------------------------------------------------
plt.figure(figsize=(8, 5))
# On trace un histogramme par niveau de gravité pour voir la répartition
severities = ["Faible", "Moyenne", "Élevée", "Critique"]
for sev in severities:
    subset = df.loc[df["Gravité"] == sev, "CVSS_num"].dropna()
    plt.hist(subset, bins=np.arange(0, 10.5, 0.5), alpha=0.6, label=sev)

plt.title("Distribution des vulnérabilités par gravité (CVSS)")
plt.xlabel("Score CVSS")
plt.ylabel("Nombre de vulnérabilités")
plt.legend()
plt.show()

# -----------------------------------------------------------------------------
# B. Diagramme circulaire des types de vulnérabilités (CWE)
# -----------------------------------------------------------------------------
top_n = 10
cwe_counts = (
    df["CWE"]
    .fillna("Inconnu")
    .value_counts()
    .nlargest(top_n)
)
plt.figure(figsize=(7, 7))
cwe_counts.plot(kind="pie", autopct="%1.1f%%", startangle=90, legend=False)
plt.title(f"Top {top_n} catégories CWE les plus fréquentes")
plt.ylabel("")
plt.show()

# -----------------------------------------------------------------------------
# C. Courbe / histogramme des scores EPSS
# -----------------------------------------------------------------------------
plt.figure(figsize=(8, 5))
df["EPSS_num"].dropna().hist(bins=20)
plt.title("Distribution des scores EPSS")
plt.xlabel("Score EPSS")
plt.ylabel("Nombre de vulnérabilités")
plt.show()

# -----------------------------------------------------------------------------
# D. Classement des éditeurs les plus affectés
# -----------------------------------------------------------------------------
top_vendors = (
    df["Éditeur"]
    .fillna("Inconnu")
    .value_counts()
    .head(10)
)
plt.figure(figsize=(9, 5))
top_vendors.plot(kind="bar")
plt.title("Top 10 des éditeurs les plus affectés")
plt.xlabel("Éditeur")
plt.ylabel("Nombre de vulnérabilités")
plt.xticks(rotation=45, ha="right")
plt.show()

# -----------------------------------------------------------------------------
# E. Heatmap CVSS vs EPSS
# -----------------------------------------------------------------------------
corr = df[["CVSS_num", "EPSS_num"]].dropna().corr()
plt.figure(figsize=(4, 4))
plt.imshow(corr, vmin=-1, vmax=1)
plt.title("Corrélation CVSS ↔ EPSS")
plt.xticks([0, 1], ["CVSS", "EPSS"])
plt.yticks([0, 1], ["CVSS", "EPSS"])
for (i, j), val in np.ndenumerate(corr.values):
    plt.text(j, i, f"{val:.2f}", ha="center", va="center", color="black")
plt.colorbar()
plt.show()

# -----------------------------------------------------------------------------
# F. Nuage de points CVSS vs EPSS
# -----------------------------------------------------------------------------
plt.figure(figsize=(8, 6))
plt.scatter(df["CVSS_num"], df["EPSS_num"], alpha=0.6)
plt.title("Nuage de points : CVSS vs EPSS")
plt.xlabel("Score CVSS")
plt.ylabel("Score EPSS")
plt.grid(True)
plt.show()

# -----------------------------------------------------------------------------
# G. Courbe cumulative du nombre de vulnérabilités dans le temps
# -----------------------------------------------------------------------------
# Si vous avez déjà une colonne de date, remplacez 'Date' par son nom
if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    timeline = (
        df.set_index("Date")
          .resample("W")
          .size()
          .cumsum()
    )
    plt.figure(figsize=(9, 5))
    timeline.plot()
    plt.title("Cumul des vulnérabilités dans le temps (par semaine)")
    plt.xlabel("Date")
    plt.ylabel("Total cumulé")
    plt.grid(True)
    plt.show()
else:
    print("⚠️  Aucune colonne de date disponible ; courbe temporelle ignorée.")

# -----------------------------------------------------------------------------
# H. Boxplot des scores CVSS par éditeur (Top 5)
# -----------------------------------------------------------------------------
top5_vendors = df["Éditeur"].value_counts().head(5).index
subset = df[df["Éditeur"].isin(top5_vendors)]
plt.figure(figsize=(10, 6))
subset.boxplot(column="CVSS_num", by="Éditeur", grid=False)
plt.title("Dispersion des scores CVSS par éditeur (Top 5)")
plt.suptitle("")  # supprime le titre auto
plt.xlabel("Éditeur")
plt.ylabel("Score CVSS")
plt.show()


KeyError: 'CVSS'